In [24]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
import pandas as pd
import numpy as np
from datetime import datetime
import time

import warnings
warnings.filterwarnings("ignore")

In [25]:
data = pd.read_csv('./allcf4_group_EDD.csv', delimiter='|')
data = data.rename(columns=lambda x: x.strip())
data = data.set_index('PGC')

data.head()

,DM_av,eDM_av,Vcmb,SGL,SGB,DM_SNIa,eDM_SNIa,DM_FP,eDM_FP,DM_TFR,eDM_TFR,DM_SBFm,eDM_SBFm,DM_SBFir,eDM_SBFir,DM_SNII,eDM_SNII
PGC,,,,,,,,,,,,,,,,,
12,35.143,0.41,6179,286.4249,11.3510,NaN,NaN,NaN,NaN,34.97,0.41,NaN,NaN,NaN,NaN,NaN,NaN
14,36.243,0.38,11474,300.8070,15.3909,NaN,NaN,NaN,NaN,36.07,0.38,NaN,NaN,NaN,NaN,NaN,NaN
16,34.803,0.45,5354,287.6120,11.7030,NaN,NaN,NaN,NaN,34.63,0.45,NaN,NaN,NaN,NaN,NaN,NaN
55,34.393,0.39,4466,327.0996,19.7763,NaN,NaN,NaN,NaN,34.22,0.39,NaN,NaN,NaN,NaN,NaN,NaN
63,35.393,0.41,8404,321.4869,19.2652,NaN,NaN,NaN,NaN,35.22,0.41,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
cc = []
for c in data.columns[:3]:
    cc.append(c)
cc += ["RA", "DEC", "GL", "GB"]
for c in data.columns[3:]:
    cc.append(c)

cc

['DM_av',
 'eDM_av',
 'Vcmb',
 'RA',
 'DEC',
 'GL',
 'GB',
 'SGL',
 'SGB',
 'DM_SNIa',
 'eDM_SNIa',
 'DM_FP',
 'eDM_FP',
 'DM_TFR',
 'eDM_TFR',
 'DM_SBFm',
 'eDM_SBFm',
 'DM_SBFir',
 'eDM_SBFir',
 'DM_SNII',
 'eDM_SNII']

In [35]:
from kapteyn import wcs

super2celestial = wcs.Transformation("supergalactic j2000 j2000", "equatorial")
# RA, DEC = ((sgl, sgb))
super2galactic = wcs.Transformation("supergalactic j2000 j2000", "galactic")
# gl, gb = tran((sgl, sgb))

In [36]:
data["RA"] = data.apply(lambda x: super2celestial((x.SGL,x.SGB))[0], axis=1)
data["DEC"] = data.apply(lambda x: super2celestial((x.SGL,x.SGB))[1], axis=1)

data["GL"] = data.apply(lambda x: super2galactic((x.SGL,x.SGB))[0], axis=1)
data["GB"] = data.apply(lambda x: super2galactic((x.SGL,x.SGB))[1], axis=1)

In [37]:
data[cc].head()

,DM_av,eDM_av,Vcmb,RA,DEC,GL,GB,SGL,SGB,DM_SNIa,...,DM_FP,eDM_FP,DM_TFR,eDM_TFR,DM_SBFm,eDM_SBFm,DM_SBFir,eDM_SBFir,DM_SNII,eDM_SNII
PGC,,,,,,,,,,,,,,,,,,,,,
12,35.143,0.41,6179,0.035972,-6.373908,90.192153,-65.929994,286.4249,11.3510,NaN,...,NaN,NaN,34.97,0.41,NaN,NaN,NaN,NaN,NaN,NaN
14,36.243,0.38,11474,359.980535,8.184610,101.660038,-52.546451,300.8070,15.3909,NaN,...,NaN,NaN,36.07,0.38,NaN,NaN,NaN,NaN,NaN,NaN
16,34.803,0.45,5354,0.047049,-5.158708,91.600561,-64.865477,287.6120,11.7030,NaN,...,NaN,NaN,34.63,0.45,NaN,NaN,NaN,NaN,NaN,NaN
55,34.393,0.39,4466,0.155821,33.600952,110.949487,-28.085699,327.0996,19.7763,NaN,...,NaN,NaN,34.22,0.39,NaN,NaN,NaN,NaN,NaN,NaN
63,35.393,0.41,8404,359.993820,28.288307,109.376188,-33.225179,321.4869,19.2652,NaN,...,NaN,NaN,35.22,0.41,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
data[cc].to_csv("tmp.csv", sep='|')
table   = np.genfromtxt('tmp.csv' , delimiter='|', 
                        filling_values=-10000, names=True, dtype=None, encoding=None)

colnames = table.dtype.names

## table is a structured array
myTable = {}
for name in table.dtype.names:
    myTable[name] = table[name]
table = myTable
## table is now a dictionary

myTable = Table()

for key in colnames:
    if key in ["SNIa"]:
        myTable.add_column(Column(data=table[key], name=key))
    elif key in ["PGC", "PGC1", "IDT14g", '1PGC']:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int))) 
    elif key in ["Vcmb", "Vmod"]:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int)))
    elif key in ["SGL", "SGB", "RA", "DEC", "GL", "GB"]:
        myTable.add_column(Column(data=table[key], name=key, format='%0.4f'))
    else:
        myTable.add_column(Column(data=table[key], name=key, format='%0.3f'))

## to be used on EDD
myTable.write('allcf4_group_bar_', format='ascii.fixed_width',delimiter='|', bookend=False, overwrite=True)